# Защита персональных данных

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится. Предсказания будут такие же.

**Обоснование:**
Формулировки для решения задания:

1. $AB \neq BA$
2. $ (AB)^{-1} = B^{-1}A^{-1} $
3. $ (AB)^{T} = B^{T}A^{T} $
4. $ AA^{-1} = E $
5. $ АЕ = ЕА = А $

 - Заменим исходную матрицу X на преобразованную матрицу XP в формуле предсказания и формуле обучения. 
 - В результате преобразования матрица P сократится до единичной. 
$$
a = XP ((X P)^T X P)^{-1} (X P)^T y = XP (P^T X^T X P)^{-1} P^T X^T y = 𝑋 (X^T X)^{-1} X^T y =Xw
$$

Как видно, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу.

## 3. Алгоритм преобразования

**Алгоритм**

1. Выделяем признаки для обучения модели и целевую переменную.
2. Преобразуем признаки и целевую переменную в массивы.
3. Создаем случайную квадратную обратимую матрицу размером равным количеству признаков - $P$. Проверим ее на обратимость.
4. Умножаем матрицу признаков на матрицу преобразований.
5. Разделяем выборку на тренировочную и тестовую.
6. Обучаем модель на тренировочной выборке и получаем предсказания на тестовой.
7. Сравниваем значения метрики R2 до и после преобразования.

**Обоснование**

Для шифрования воспользуемся квадратной обратимой матрицей - $P$, где $Xcoding =XP$ - процесс шифрования. В п.2 обосновал возможность использования матрицы $P$ для шифрования исходной матрицы признаков. 

## 4. Проверка алгоритма

In [ ]:
def algorithm(data):
    features = data.drop(['Страховые выплаты'], axis = 1)
    target = data['Страховые выплаты']
    features = features.values
    target = target.values  
    
    good = 0
    while good != 1:
        P = np.random.randint(0, 10, (features.shape[1], features.shape[1]))
        try:
            P_invert = np.linalg.inv(P)
            good = 1  
        except np.linalg.LinAlgError:
            continue
    
    X = features.dot(P)
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

    display('Данные до преобразования')
    display(features)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    display('Метрика r2 без преобразования')
    display(r2_score(target_valid, predict))
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    X, target, test_size=0.25, random_state=12345)
    
    display('Данные после преобразования')
    display(X)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    display('Метрика r2 с преобразованием')
    display(r2_score(target_valid, predict))

In [ ]:
algorithm(data)


'Данные до преобразования'

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

'Метрика r2 без преобразования'

0.435227571270266

'Данные после преобразования'

array([[4.30000e+01, 3.97016e+05, 2.48049e+05, 3.85000e+02],
       [4.70000e+01, 3.04239e+05, 1.90052e+05, 4.21000e+02],
       [2.90000e+01, 1.68145e+05, 1.05029e+05, 2.61000e+02],
       ...,
       [2.20000e+01, 2.71318e+05, 1.69532e+05, 1.94000e+02],
       [2.60000e+01, 2.61739e+05, 1.63542e+05, 2.28000e+02],
       [3.00000e+01, 3.24951e+05, 2.03036e+05, 2.68000e+02]])

'Метрика r2 с преобразованием'

0.4352275712702275

 - В результате преобразования видим, что данные признаков потеряли свои оригинальные значения. Метрика при этом сильно не поменялась.

## Вывод


1. Загрузил и изучил данные.
2. Проверил, что качество линейной регресии в результате умножения признаков на обртную матрицу не изменилось.
3. Создал алгоритм преобразования данных.
4. Исследовал алгоритм преобразования данных и проверил метрику R2 для данных без преобразования и с ним.

В результате работы алгоритма данные преобразуются так, что персональную информацию по ним уже не восстановить. Качество линейной регрессии при этом остается на прежнем уровне.